In [ ]:
%%capture
import ROOT
import glob
import math
import sys
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.drawMultiTrending import *
from TPCQCVis.src.checkHistograms import *
from TPCQCVis.src.checkTrending import *
from TPCQCVis.src.utility import *
from TPCQCVis.src.drawBetheBloch import * 
import warnings
from copy import copy
warnings.filterwarnings('ignore')

In [ ]:
%jsroot on
display(HTML("<style>.container { width:95% !important; align-items: center;}</style>"))
display(HTML("<style>table {float:left;}</style>"))
ROOT.gErrorIgnoreLevel = ROOT.kFatal
#display(HTML('<style>{}</style>'.format(CSS)))
#ROOT.gStyle.SetPalette(57)

In [ ]:
periodName = "LHC23zzh"
passName = "apass3"
runNumber = "544116"
path = "/cave/alice-tpc-qc/data/2023/"

# Read QC root file
rootDataFile=[]
rootDataFile.append(ROOT.TFile.Open(path+"/"+periodName+"/"+passName+"/"+str(runNumber)+"_QC.root","READ"))

# dEdx vs p

In [ ]:
betheBlochParams = [0.21226799488067627,4.191800117492676,0.010126300156116486,2.278209924697876,0.9807400107383728]
bBLines = getBetheBlochLines(betheBlochParams)
objects = ["hdEdxTotVsP_TPC"]

hists = []
for i,objectName in enumerate(objects):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",log="logxyz",yAxisRange=[10,6000])
    hists.append(copy(hist[0]))
c = ROOT.TCanvas("TPC_dEdx_vs_p","TPC_dEdx_vs_p",800,700)
#c.Divide(2)
# TPC
c.cd()
hists[0].Rebin(1)
hists[0].Draw("COLZ")
hists[0].SetStats(0)
drawBetheBloch([bBLines],c)
legend = ROOT.TLegend(0.55,0.65,0.87,0.85)
legend.AddEntry(bBLines[0],"ALEPH Bethe-Bloch fit")
legend.SetHeader("#splitline{#bf{LHC23zzh apass3 544116}}{nClusters > 60, |tan(#lambda)| < 1}","C")
legend.Draw()

ROOT.gPad.SetLogx(1)
ROOT.gPad.SetLogy(1)
ROOT.gPad.SetLogz(1)

c.Draw()

In [ ]:
c.Draw()

In [ ]:
c.SaveAs("TPC_dEdx_vs_p.pdf");

# DCAR Phi

In [ ]:
%jsroot on
myFitMeans = []
myFitRMSs = []
myPads = []
myHists = []
for objectName in ["hDCAr_A_Pos_pTmin","hDCAr_C_Pos_pTmin","hDCAr_A_Neg_pTmin","hDCAr_C_Neg_pTmin"]:
    if checkIfExists(rootDataFile,objectName):
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5],size=[1200,400])
        #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
        pad1.cd()
        canvas.SetCanvasSize(1000,400)
        leg = ROOT.TLegend()
        hist[0].FitSlicesY()
        fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
        fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
        fitMean.SetMarkerStyle(21)
        fitMean.SetMarkerSize(0.4)
        fitMean.SetMarkerColor(1)
        fitMean.Draw("SAME")
        fitRMS.SetMarkerStyle(25)
        fitRMS.SetMarkerSize(0.4)
        fitRMS.SetMarkerColor(2)
        fitRMS.Draw("SAME")
        leg.AddEntry(fitMean,"Mean")
        leg.AddEntry(fitRMS,"RMS")
        leg.Draw()
        myHists.append(copy(hist[0]))
        myFitMeans.append(copy(fitMean))
        myFitRMSs.append(copy(fitRMS))
        myPads.append(copy(pad1))

In [ ]:
bigC = ROOT.TCanvas("TPC_DCAR_vs_phi","TPC_DCAR_vs_phi",1200,600)
bigC.Divide(2,2,0.01,0.02)
legends = []
titles = ["#bf{A-Side Positive Tracks}",
          "#bf{C-Side Positive Tracks}",
          "#bf{A-Side Negative Tracks}",
          "#bf{C-Side Negative Tracks}",]
for i in range(len(myHists)): 
    pad = bigC.cd(i+1)
    if i == 0 :   pad.SetMargin(0.15, 0.0, 0.0, 0.15)
    elif i == 1 : pad.SetMargin(0.0, 0.15, 0.0, 0.15)
    elif i == 2 : pad.SetMargin(0.15, 0.0, 0.15, 0.0)
    elif i == 3 : pad.SetMargin(0.0, 0.15, 0.15, 0.0)
    
    myHists[i].Draw("COL")
    myFitMeans[i].Draw("SAME")
    myFitRMSs[i].Draw("SAME")
    
    legend = ROOT.TLegend(0.7-pad.GetRightMargin(),0.70-pad.GetTopMargin(),0.97-pad.GetRightMargin(),0.95-pad.GetTopMargin())
    legend.SetHeader(titles[i],"C")
    legend.AddEntry(myFitMeans[i],"Mean")
    legend.AddEntry(myFitRMSs[i],"RMS")
    legends.append(copy(legend))
    
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorBoundary.SetLineColor(ROOT.kWhite)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kWhite)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-5,(j*2*math.pi)/18,5)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,-2.5,str(j))
    
    targets = [ROOT.TLine(hist[0].GetXaxis().GetXmin(),val,hist[0].GetXaxis().GetXmax(),val) for val in [-0.2,0,0.2]]
    for l,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if l == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()

    myHists[i].SetTitle("")
    legends[i].Draw()

bigC.cd()
latex = ROOT.TLatex(0.38,0.95,"#splitline{LHC23zzh apass3 544116}{#bf{nClusters > 60, |tan(#lambda)| < 1, pT > 1 GeV}}")
latex.SetTextSize(0.03)
latex.SetTextAlign(12)
latex.Draw()
bigC.Draw()

In [ ]:
bigC.SaveAs("TPC_DCAR.pdf");

In [ ]:
with ROOT.TFile("TPC_QC_plots.root","RECREATE") as myFile:
    myFile.cd()
    c.Write()
    bigC.Write()
    myFile.Write()